In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

### Importing the necessary libraries

In [ ]:
import warnings 
warnings.filterwarnings('ignore')

In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style('darkgrid')
cmap = sns.cm.mako_r

%matplotlib inline

from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

### Importing the Data using Pandas

In [ ]:
aug_train = pd.read_csv('../input/hr-analytics-job-change-of-data-scientists/aug_train.csv')

In [ ]:
aug_train.head()

In [ ]:
aug_train.info()

In [ ]:
aug_train.isna().sum()

As we can see that there are a lot of missing values and most of them are Categorical and it is not feasible to fill those values as we may end up providing completly wrong data to the training set. So, I prefer dropping them instead.

### Drop NA values

In [ ]:
aug_train.dropna(inplace=True)

## EDA

In [ ]:
# Function to Plot Bar Graph
def plot_bar_graph(column, order=None):
    x = aug_train[column].value_counts().index
    y = aug_train[column].value_counts().values
    sns.barplot(x, y, order=order)
     
# Funtion to Plot Pie Chart
def plot_pie_chart(column, title=''):
    values = aug_train[column].value_counts().values
    labels = aug_train[column].value_counts().index
    plt.figure(figsize=(6,6))
    plt.title(title)
    plt.pie(x=values, labels=labels, autopct='%1.2f%%')

### Pie Chart for Gender

In [ ]:
plot_pie_chart('gender', title='Pie Chart For Gender')
plt.show()

As we can see that most of the Candidates working are Male

### Pie Chart for Relevent Experience

In [ ]:
plot_pie_chart('relevent_experience', title='Pie Chart For Relevent Experience')
plt.show()

### Pie Chart for Enrolled University

In [ ]:
plot_pie_chart('enrolled_university', title='Pie Chart For Enrolled University')
plt.show()

### Pie Chart Education Level

In [ ]:
plot_pie_chart('education_level', title='Pie Chart For Education Level')
plt.show()

### Bar chart for Major Discipline

In [ ]:
plot_bar_graph('major_discipline')
plt.show()

Most of the Candidates are from STEM. That is their major discipline was in one of the Following:

Science,
Technology,
Engineering and
Mathematics

### Bar chart for Company Size

In [ ]:
plt.figure(figsize=(8, 4))
order = ['<10', '10/49', '50-99', '100-500', '500-999', '1000-4999', '5000-9999', '10000+']
plot_bar_graph('company_size', order=order)
plt.show()

### Bar chart for Company Type

In [ ]:
plt.figure(figsize=(9, 5))
plot_bar_graph('company_type')
plt.show()

### How many Jobs Candidates Worked before joining the current company

In [ ]:
plt.figure(figsize=(9, 5))
order = ['never', '1', '2', '3', '4', '>4']
plot_bar_graph('last_new_job', order=order)
plt.show()

### Let's play a Histogram to see the Distribution of the Training Hours

In [ ]:
plt.figure(figsize=(12, 6))
sns.histplot(aug_train['training_hours'], kde=True)

The graph reveals the most of the Candidates train for 20 to 50 Hours

Before moving on to the prediction part we will check relation between few features and Training Hours, as people looking for a Job Change tend to Train for longer hours.
Relevent Experience vs Training Hours

Education Level vs Training Hours

Major Discipline vs Training Hours

Company Size vs Training Hours

Company Type vs Training Hours

Last New Job vs Training Hours

In [ ]:
figures, axes = plt.subplots(3, 2, figsize=(16, 14))

sns.scatterplot('relevent_experience', 'training_hours', data=aug_train, hue='target', ax=axes[0,0])
sns.scatterplot('education_level', 'training_hours', data=aug_train, hue='target', ax=axes[0,1])
sns.scatterplot('major_discipline', 'training_hours', data=aug_train, hue='target', ax=axes[1,0])
sns.scatterplot('company_size', 'training_hours', data=aug_train, hue='target', ax=axes[1,1])
sns.scatterplot('company_type', 'training_hours', data=aug_train, hue='target', ax=axes[2,0])
sns.scatterplot('last_new_job', 'training_hours', data=aug_train, hue='target', ax=axes[2,1])

plt.show()

## Data Preprocessing Phase

In [ ]:
# Using pandas get_dummies() to convert the Categorical Data to Numerical Features
# Then Removing the Unnecessary Columns

list_of_columns = ['city', 'gender', 'relevent_experience', 'enrolled_university', 'education_level',
                   'major_discipline', 'experience', 'company_size', 'company_type', 'last_new_job']

aug_train_dummies_df = []

for col in list_of_columns:
    dummy_train_df = pd.get_dummies(aug_train[col])
    aug_train_dummies_df.append(dummy_train_df)



list_of_columns.append('enrollee_id')    

aug_train_dummies_df.insert(0, aug_train.drop(columns=list_of_columns))

aug_train = pd.concat(aug_train_dummies_df,axis=1)

### Dataset split

In [ ]:
X = aug_train.dropna().drop(columns=['target']).values
y = aug_train.dropna()['target'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=80)

## Model Pipeline

In [ ]:
pipeline = make_pipeline(StandardScaler(), SVC(kernel='sigmoid'))
pipeline.fit(X_train, y_train)
prediction = pipeline.predict(X_test)

print(f'Accuracy of the Model is {accuracy_score(y_test, prediction)}')

###  Confussion Matrix

In [ ]:
conf_matrix = confusion_matrix(y_test, prediction)
sns.heatmap(conf_matrix, linewidths=1, cmap=cmap, annot=True)
plt.plot()